# Script to update the "unplugged" GBR models and to produce extended climate timeseries for input into other GBR models


### Unplugged Source models
* The unplugged Source models are located on Athena here: \\athena\pdreef\Source_Modelling\Climate_extension\Source_models
* Read this document for information on the types of unplugged models: \\athena\pdreef\Source_Modelling\Climate_extension\Source_models\READ_ME_Origin_of_Source_models.docx

### Climate storage relationships
* The climate storage relationships can be found here: \\athena\pdreef\Source_Modelling\Climate_extension\Storage_climate_sources

## Script information
This script uses a "dummy" Source model to generate climate timeseries that can be used to extend the climate on a Source model of your choice. 

You will need to have veneer-py and the Veneer plugin installed to use this script. Refer https://github.com/flowmatters/veneer-py/blob/master/doc/training/0Setup_and_Hello_World.ipynb to for instructions on how to install Veneer.

### Before running this script!!
* Open a generic Source model that has the DS plugin and functionality. Note this model does not need to be the model you are intending to extend. It can be a much smaller model, for example, the Beckers model (\\athena\pdreef\Source_Modelling\Beckers\Beckers_GBR_411_Seasonal.rsproj)
* In the model, go to the 'Tools' menu, select 'Web Server Monitoring'. Choose a port number, check 'Allow Scripts' and start the server connection.

In [1]:
import veneer
import os
import pandas as pd
import numpy as np
import sys

import dsed
from dsed import preprocessors

from datetime import datetime,timedelta

import time
import random

then = time.time() # Time before the climate collation occurs

C:\ProgramData\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


climate Dynamic_SedNet.Parameterisation.Models.ClimateCollationModel
gully Dynamic_SedNet.Parameterisation.Models.GullyParameterisationModel
usle Dynamic_SedNet.Parameterisation.Models.CoverTimeSeries_SpatialPreprocessorModel
gbrusle GBR_DynSed_Extension.Parameterisation.Models.GBRUSLECoverTimeSeriesSpatialPreprocessorModel
lewis GBR_DynSed_Extension.Parameterisation.Models.LewisTrappingParameterisationModel


### Set the port number below according to the one that is active in your 'Web Server Monitoring' window

In [2]:
PORT=9876
v = veneer.Veneer(port=PORT)

In [3]:
veneer.general.PRINT_SCRIPTS=False

## Comment out the regions you do not wish to process data for

In [4]:
## Comment out the regions below that you do not wish to process data for

REGIONs = ['BU_without_SC1',
           'WT_noScaling',
          'WT_inclScaling',
           'CY',
           'MW',
           'BM',
           'FI'
          ]

In [5]:
%xmode Verbose

Exception reporting mode: Verbose


## Climate inputs (SILO download)
* Climate data downloaded from SILO (http://siloprod/silo/spatial.html) for the extents specified in: \\athena\pdreef\Source_Modelling\Model_Inputs\ClimateInputs\SILOdumps\extent_rectangle.xls

## Settings
Check and change the file paths below as needed.

#### Convert the netCDF SILO files to .tif using the script "ConvertSILONetCDFtoTIFs" and put in folders rain_grids and pet_grids under the same directory 'local_climate_dir' in the cell below.

In [6]:
###### Set the start and end date of your required climate TS (typically is the period of your downloaded SILO climate files)


#StartDate = '01/01/2018'
#EndDate = '30/06/2021'

StartDate = '01/07/2021'
EndDate = '30/11/2021'


#local_climate_dir = r'E:\ClimateExtensions\Rob_2020_Aug2021\TIFs'
local_climate_dir = r'E:\ClimateExtensions\Rob_Nov2021\TIFs'


In [7]:
# Set up the local outputTS directory from which output folders for each region will be created. 
#outputTS_dir = r'E:\ClimateExtensions\Rob_2020_Aug2021\ModelCSVs'
outputTS_dir = r'E:\ClimateExtensions\Rob_Nov2021\ModelCSVs'

if not os.path.exists(outputTS_dir):
    os.makedirs(outputTS_dir)
    print("Made dir: " + outputTS_dir)

# Create new output TS folders for each region
os.makedirs(outputTS_dir +'\\'+ 'BU_without_SC1', exist_ok=True)
os.makedirs(outputTS_dir +'\\'+'WT_noScaling', exist_ok=True)
os.makedirs(outputTS_dir +'\\'+'WT_inclScaling', exist_ok=True)
os.makedirs(outputTS_dir +'\\'+'CY', exist_ok=True)
os.makedirs(outputTS_dir +'\\'+'MW', exist_ok=True)
os.makedirs(outputTS_dir +'\\'+'FI', exist_ok=True)
os.makedirs(outputTS_dir +'\\'+'BM', exist_ok=True)

In [8]:
climcoords_path = r'E:\ClimateExtensions\CatchModCSVs'
subCatchmentPolys_path = r'E:\ClimateExtensions\CatchModCSVs\catchmentShapes'

In [9]:
REGIONs

['BU_without_SC1', 'WT_noScaling', 'WT_inclScaling', 'CY', 'MW', 'BM', 'FI']

## Script that loops through the regions specified at the start of the notebook to produce the PET and Rain files for each SC

In [10]:
# Specifies the required files and parameters to run the climate preprocessor for each of the regions

rainDIR = 'daily_rain'
petDIR = 'et_morton_wet'


for region in REGIONs:
    if region=='BU_with_SC1':
        rainfallInputDirectory = local_climate_dir + '\\' + rainDIR
        petInputDirectory = local_climate_dir + '\\' + petDIR
        subCatchmentPolys = 'BU\\Exported ReportingRegions.shp'
        subCatchFieldName = '"Catchmt"'
        outputDIR = os.path.join(outputTS_dir + '\\','BU_with_SC1')
        geoCoordFileName = 'BU_RC8_30m_climcoords_IncludingSC1.csv'
        rainScalingFile = '""'
        parameters = {
            'updateModels':False,
            'useInternalSubCatchmentPolys':False,
            'Path_SubCatchPolys':'"' + subCatchmentPolys_path + '\\' + subCatchmentPolys +'"',
            'SubCatchPolys_FieldName':subCatchFieldName,
            'OutputDirectory':'"'+ outputDIR +'"',
            'coordsFileName':'"'+ climcoords_path + '\\' + geoCoordFileName +'"',
            'rainfallInputDirectory':'"' + rainfallInputDirectory + '"',
            'rainfallDatePattern':'"yyyymmdd"',
            'petInputDirectory':'"' + petInputDirectory + '"',
            'petDatePattern':'"yyyymmdd"',
            'SaveTimeSeries':True,
            'TSonly':False,    
            'useExistingGeoCoords':True,
            'saveCoordsFile':False,    
            'projectionString':'"Australian Albers"',
            'cellSize':30,
            'appendToExisting':False,
            'StartDateString':'"' + StartDate + '"',
            'EndDateString':'"' + EndDate + '"',
            'Path_RainScalingFile':rainScalingFile,
            'valueRasterTypeStr':'"GeoTiff .tif"'
        }
        
        preprocessor = preprocessors.run_climate
        UNUSED = []
        defaults = preprocessors.climate_default_params(v)
        {k:v for k,v in defaults.items() if v is None and not k in parameters and not k in UNUSED}
        veneer.general.PRINT_SCRIPTS=True
        result = preprocessors.run_climate(v,**parameters)

        
    elif region=='BU_without_SC1':
        rainfallInputDirectory = local_climate_dir + '\\' + rainDIR
        petInputDirectory = local_climate_dir + '\\' + petDIR
        subCatchmentPolys = 'BU\\Exported ReportingRegions.shp' # -- Need the shapefile for BU without SC#1!!
        subCatchFieldName = '"Catchmt"'
        outputDIR = os.path.join(outputTS_dir + '\\', 'BU_without_SC1')
        geoCoordFileName = 'BU_RC8_30m_climcoords_NoSC1.csv'
        rainScalingFile = '""'
        parameters = {
            'updateModels':False,
            'useInternalSubCatchmentPolys':False,
            'Path_SubCatchPolys':'"' + subCatchmentPolys_path + '\\' + subCatchmentPolys +'"',
            'SubCatchPolys_FieldName':subCatchFieldName,
            'OutputDirectory':'"'+ outputDIR +'"',
            'coordsFileName':'"'+ climcoords_path + '\\' + geoCoordFileName +'"',
            'rainfallInputDirectory':'"' + rainfallInputDirectory + '"',
            'rainfallDatePattern':'"yyyymmdd"',
            'petInputDirectory':'"' + petInputDirectory + '"',
            'petDatePattern':'"yyyymmdd"',
            'SaveTimeSeries':True,
            'TSonly':False,    
            'useExistingGeoCoords':True,
            'saveCoordsFile':False,    
            'projectionString':'"Australian Albers"',
            'cellSize':30,
            'appendToExisting':False,
            'StartDateString':'"' + StartDate + '"',
            'EndDateString':'"' + EndDate + '"',
            'Path_RainScalingFile':rainScalingFile,
            'valueRasterTypeStr':'"GeoTiff .tif"'
        }
        
        preprocessor = preprocessors.run_climate
        UNUSED = []
        defaults = preprocessors.climate_default_params(v)
        {k:v for k,v in defaults.items() if v is None and not k in parameters and not k in UNUSED}
        veneer.general.PRINT_SCRIPTS=True
        result = preprocessors.run_climate(v,**parameters)
        

    elif region=='WT_noScaling':
        rainfallInputDirectory = local_climate_dir + '\\' + rainDIR
        petInputDirectory = local_climate_dir + '\\' + petDIR
        subCatchmentPolys = 'WT\\Exported ReportingRegions.shp'
        subCatchFieldName = '"Catchmt"'
        outputDIR = os.path.join(outputTS_dir + '\\','WT_noScaling')
        geoCoordFileName = 'WT_RC8_30m_climcoords.csv'
        rainScalingFile = '""'
#         rainScalingFile = '"' + os.path.join(climcoords_path,'wt_rain_scaling_factors.csv') + '"' 
        parameters = {
            'updateModels':False,
            'useInternalSubCatchmentPolys':False,
            'Path_SubCatchPolys':'"' + os.path.join(subCatchmentPolys_path + '\\',subCatchmentPolys) +'"',
            'SubCatchPolys_FieldName':subCatchFieldName,
            'OutputDirectory':'"'+ outputDIR +'"',
            'coordsFileName':'"'+ os.path.join(climcoords_path + '\\', geoCoordFileName) +'"',
            'rainfallInputDirectory':'"' + rainfallInputDirectory + '"',
            'rainfallDatePattern':'"yyyymmdd"',
            'petInputDirectory':'"' + petInputDirectory + '"',
            'petDatePattern':'"yyyymmdd"',
            'SaveTimeSeries':True,
            'TSonly':False,    
            'useExistingGeoCoords':True,
            'saveCoordsFile':False,    
            'projectionString':'"Australian Albers"',
            'cellSize':30,
            'appendToExisting':False,
            'StartDateString':'"' + StartDate + '"',
            'EndDateString':'"' + EndDate + '"',
            'Path_RainScalingFile':rainScalingFile,
            'valueRasterTypeStr':'"GeoTiff .tif"'
        }
        
        preprocessor = preprocessors.run_climate
        UNUSED = []
        defaults = preprocessors.climate_default_params(v)
        {k:v for k,v in defaults.items() if v is None and not k in parameters and not k in UNUSED}
        veneer.general.PRINT_SCRIPTS=True
        result = preprocessors.run_climate(v,**parameters)
        
        
    elif region=='WT_inclScaling':
        rainfallInputDirectory = local_climate_dir + '\\' + rainDIR
        petInputDirectory = local_climate_dir + '\\' + petDIR
        subCatchmentPolys = 'WT\\Exported ReportingRegions.shp'
        subCatchFieldName = '"Catchmt"'
        outputDIR = os.path.join(outputTS_dir + '\\','WT_inclScaling')
        geoCoordFileName = 'WT_RC8_30m_climcoords.csv'
        rainScalingFile = '"' + os.path.join(climcoords_path,'WT_scaling_new_includingTullyEuramo.csv') + '"' 
        parameters = {
            'updateModels':False,
            'useInternalSubCatchmentPolys':False,
            'Path_SubCatchPolys':'"' + os.path.join(subCatchmentPolys_path + '\\',subCatchmentPolys) +'"',
            'SubCatchPolys_FieldName':subCatchFieldName,
            'OutputDirectory':'"'+ outputDIR +'"',
            'coordsFileName':'"'+ os.path.join(climcoords_path + '\\', geoCoordFileName) +'"',
            'rainfallInputDirectory':'"' + rainfallInputDirectory + '"',
            'rainfallDatePattern':'"yyyymmdd"',
            'petInputDirectory':'"' + petInputDirectory + '"',
            'petDatePattern':'"yyyymmdd"',
            'SaveTimeSeries':True,
            'TSonly':False,    
            'useExistingGeoCoords':True,
            'saveCoordsFile':False,    
            'projectionString':'"Australian Albers"',
            'cellSize':30,
            'appendToExisting':False,
            'StartDateString':'"' + StartDate + '"',
            'EndDateString':'"' + EndDate + '"',
            'Path_RainScalingFile':rainScalingFile,
            'valueRasterTypeStr':'"GeoTiff .tif"'
        }
        
        preprocessor = preprocessors.run_climate
        UNUSED = []
        defaults = preprocessors.climate_default_params(v)
        {k:v for k,v in defaults.items() if v is None and not k in parameters and not k in UNUSED}
        veneer.general.PRINT_SCRIPTS=True
        result = preprocessors.run_climate(v,**parameters)
        
    elif region=='CY':
        rainfallInputDirectory = local_climate_dir + '\\' + rainDIR
        petInputDirectory = local_climate_dir + '\\' + petDIR
        subCatchmentPolys = 'CY\\Exported ReportingRegions.shp'
        subCatchFieldName = '"Catchmt"'
        outputDIR = os.path.join(outputTS_dir + '\\','CY')
        geoCoordFileName = 'CY_RC8_30m_climcoords.csv'
        rainScalingFile = '""'
        parameters = {
            'updateModels':False,
            'useInternalSubCatchmentPolys':False,
            'Path_SubCatchPolys':'"' + os.path.join(subCatchmentPolys_path + '\\',subCatchmentPolys) +'"',
            'SubCatchPolys_FieldName':subCatchFieldName,
            'OutputDirectory':'"'+ outputDIR +'"',
            'coordsFileName':'"'+ os.path.join(climcoords_path + '\\', geoCoordFileName) +'"',
            'rainfallInputDirectory':'"' + rainfallInputDirectory + '"',
            'rainfallDatePattern':'"yyyymmdd"',
            'petInputDirectory':'"' + petInputDirectory + '"',
            'petDatePattern':'"yyyymmdd"',
            'SaveTimeSeries':True,
            'TSonly':False,    
            'useExistingGeoCoords':True,
            'saveCoordsFile':False,    
            'projectionString':'"Australian Albers"',
            'cellSize':30,
            'appendToExisting':False,
            'StartDateString':'"' + StartDate + '"',
            'EndDateString':'"' + EndDate + '"',
            'Path_RainScalingFile':rainScalingFile,
            'valueRasterTypeStr':'"GeoTiff .tif"'
        }
        
        preprocessor = preprocessors.run_climate
        UNUSED = []
        defaults = preprocessors.climate_default_params(v)
        {k:v for k,v in defaults.items() if v is None and not k in parameters and not k in UNUSED}
        veneer.general.PRINT_SCRIPTS=True
        result = preprocessors.run_climate(v,**parameters)

        
    elif region=='MW':
        rainfallInputDirectory = local_climate_dir + '\\' + rainDIR
        petInputDirectory = local_climate_dir + '\\' + petDIR
        subCatchmentPolys = 'MW\\Exported ReportingRegions.shp'
        subCatchFieldName = '"Catchmt"'
        outputDIR = os.path.join(outputTS_dir + '\\','MW')
        geoCoordFileName = 'MW_RC8_30m_climcoords.csv'
        rainScalingFile = '""'
        parameters = {
            'updateModels':False,
            'useInternalSubCatchmentPolys':False,
            'Path_SubCatchPolys':'"' + os.path.join(subCatchmentPolys_path + '\\',subCatchmentPolys) +'"',
            'SubCatchPolys_FieldName':subCatchFieldName,
            'OutputDirectory':'"'+ outputDIR +'"',
            'coordsFileName':'"'+ os.path.join(climcoords_path + '\\', geoCoordFileName) +'"',
            'rainfallInputDirectory':'"' + rainfallInputDirectory + '"',
            'rainfallDatePattern':'"yyyymmdd"',
            'petInputDirectory':'"' + petInputDirectory + '"',
            'petDatePattern':'"yyyymmdd"',
            'SaveTimeSeries':True,
            'TSonly':False,    
            'useExistingGeoCoords':True,
            'saveCoordsFile':False,    
            'projectionString':'"Australian Albers"',
            'cellSize':30,
            'appendToExisting':False,
            'StartDateString':'"' + StartDate + '"',
            'EndDateString':'"' + EndDate + '"',
            'Path_RainScalingFile':rainScalingFile,
            'valueRasterTypeStr':'"GeoTiff .tif"'
        }
        
        preprocessor = preprocessors.run_climate
        UNUSED = []
        defaults = preprocessors.climate_default_params(v)
        {k:v for k,v in defaults.items() if v is None and not k in parameters and not k in UNUSED}
        veneer.general.PRINT_SCRIPTS=True
        result = preprocessors.run_climate(v,**parameters)
        

    elif region=='FI':
        rainfallInputDirectory = local_climate_dir + '\\' + rainDIR
        petInputDirectory = local_climate_dir + '\\' + petDIR
        subCatchmentPolys = 'FI\\Exported ReportingRegions.shp'
        subCatchFieldName = '"Catchmt"'
        outputDIR = os.path.join(outputTS_dir + '\\','FI')
        geoCoordFileName = 'FI_RC8_30m_climcoords.csv'
        rainScalingFile = '""'
        parameters = {
            'updateModels':False,
            'useInternalSubCatchmentPolys':False,
            'Path_SubCatchPolys':'"' + os.path.join(subCatchmentPolys_path + '\\',subCatchmentPolys) +'"',
            'SubCatchPolys_FieldName':subCatchFieldName,
            'OutputDirectory':'"'+ outputDIR +'"',
            'coordsFileName':'"'+ os.path.join(climcoords_path + '\\', geoCoordFileName) +'"',
            'rainfallInputDirectory':'"' + rainfallInputDirectory + '"',
            'rainfallDatePattern':'"yyyymmdd"',
            'petInputDirectory':'"' + petInputDirectory + '"',
            'petDatePattern':'"yyyymmdd"',
            'SaveTimeSeries':True,
            'TSonly':False,    
            'useExistingGeoCoords':True,
            'saveCoordsFile':False,    
            'projectionString':'"Australian Albers"',
            'cellSize':30,
            'appendToExisting':False,
            'StartDateString':'"' + StartDate + '"',
            'EndDateString':'"' + EndDate + '"',
            'Path_RainScalingFile':rainScalingFile,
            'valueRasterTypeStr':'"GeoTiff .tif"'
        }
        
        preprocessor = preprocessors.run_climate
        UNUSED = []
        defaults = preprocessors.climate_default_params(v)
        {k:v for k,v in defaults.items() if v is None and not k in parameters and not k in UNUSED}
        veneer.general.PRINT_SCRIPTS=True
        result = preprocessors.run_climate(v,**parameters)
        
        
    elif region=='BM':
        rainfallInputDirectory = local_climate_dir + '\\' + rainDIR
        petInputDirectory = local_climate_dir + '\\' + petDIR
        subCatchmentPolys = 'BM\\Exported ReportingRegions.shp'
        subCatchFieldName = '"Catchmt"'
        outputDIR = os.path.join(outputTS_dir + '\\','BM')
        geoCoordFileName = 'BM_RC8_30m_climcoords.csv'
        rainScalingFile = '""'
        rainScalingFile = '""'
        parameters = {
            'updateModels':False,
            'useInternalSubCatchmentPolys':False,
            'Path_SubCatchPolys':'"' + os.path.join(subCatchmentPolys_path + '\\',subCatchmentPolys) +'"',
            'SubCatchPolys_FieldName':subCatchFieldName,
            'OutputDirectory':'"'+ outputDIR +'"',
            'coordsFileName':'"'+ os.path.join(climcoords_path + '\\', geoCoordFileName) +'"',
            'rainfallInputDirectory':'"' + rainfallInputDirectory + '"',
            'rainfallDatePattern':'"yyyymmdd"',
            'petInputDirectory':'"' + petInputDirectory + '"',
            'petDatePattern':'"yyyymmdd"',
            'SaveTimeSeries':True,
            'TSonly':False,    
            'useExistingGeoCoords':True,
            'saveCoordsFile':False,    
            'projectionString':'"Australian Albers"',
            'cellSize':30,
            'appendToExisting':False,
            'StartDateString':'"' + StartDate + '"',
            'EndDateString':'"' + EndDate + '"',
            'Path_RainScalingFile':rainScalingFile,
            'valueRasterTypeStr':'"GeoTiff .tif"'
        }
        
        preprocessor = preprocessors.run_climate
        UNUSED = []
        defaults = preprocessors.climate_default_params(v)
        {k:v for k,v in defaults.items() if v is None and not k in parameters and not k in UNUSED}
        veneer.general.PRINT_SCRIPTS=True
        result = preprocessors.run_climate(v,**parameters)
        

# Generated Script
import Dynamic_SedNet.Parameterisation.Models.ClimateCollationModel
import clr
clr.AddReference('System.Core')
import System
import FlowMatters.Source.Veneer.RemoteScripting.ScriptHelpers as H
clr.ImportExtensions(System.Linq)
from TIME.Core.Metadata import OutputAttribute
result = []
tmp = Dynamic_SedNet.Parameterisation.Models.ClimateCollationModel()
typeObject = tmp.GetType()
for member in dir(tmp):
  try:
    if typeObject.GetMember(member)[0].IsDefined(OutputAttribute,True):
      result.append(member)
  except: pass

import Dynamic_SedNet.Parameterisation.Models.ClimateCollationModel as ClimateCollationModel
import Dynamic_SedNet.Tools.ToolsModel as ToolsModel

p = ClimateCollationModel()
p.Scenario = scenario
p.CatPolys = scenario.GeographicData.IntersectedSCFUPolygons[1]
p.FuPolys = scenario.GeographicData.IntersectedSCFUPolygons[1]
p.updateModels = False
p.useInternalSubCatchmentPolys = False
p.Path_SubCatchPolys = "E:\ClimateExtensions\CatchModCSVs\catchmen


import Dynamic_SedNet.Parameterisation.Models.ClimateCollationModel as ClimateCollationModel
import Dynamic_SedNet.Tools.ToolsModel as ToolsModel

p = ClimateCollationModel()
p.Scenario = scenario
p.CatPolys = scenario.GeographicData.IntersectedSCFUPolygons[1]
p.FuPolys = scenario.GeographicData.IntersectedSCFUPolygons[1]
p.updateModels = False
p.useInternalSubCatchmentPolys = False
p.Path_SubCatchPolys = "E:\ClimateExtensions\CatchModCSVs\catchmentShapes\WT\Exported ReportingRegions.shp"
p.SubCatchPolys_FieldName = "Catchmt"
p.OutputDirectory = "E:\ClimateExtensions\Rob_Nov2021\ModelCSVs\WT_inclScaling"
p.coordsFileName = "E:\ClimateExtensions\CatchModCSVs\WT_RC8_30m_climcoords.csv"
p.rainfallInputDirectory = "E:\ClimateExtensions\Rob_Nov2021\TIFs\daily_rain"
p.rainfallDatePattern = "yyyymmdd"
p.petInputDirectory = "E:\ClimateExtensions\Rob_Nov2021\TIFs\et_morton_wet"
p.petDatePattern = "yyyymmdd"
p.SaveTimeSeries = True
p.TSonly = False
p.useExistingGeoCoords = True
p.saveCoordsFile

# Generated Script
import Dynamic_SedNet.Parameterisation.Models.ClimateCollationModel
import clr
clr.AddReference('System.Core')
import System
import FlowMatters.Source.Veneer.RemoteScripting.ScriptHelpers as H
clr.ImportExtensions(System.Linq)
from TIME.Core.Metadata import ParameterAttribute
result = []
tmp = Dynamic_SedNet.Parameterisation.Models.ClimateCollationModel()
typeObject = tmp.GetType()
for member in dir(tmp):
  try:
    if typeObject.GetMember(member)[0].IsDefined(ParameterAttribute,True):
      result.append(member)
  except: pass

import Dynamic_SedNet.Parameterisation.Models.ClimateCollationModel
model = Dynamic_SedNet.Parameterisation.Models.ClimateCollationModel()
result = {}

try: result["EndDateString"]=model.EndDateString
except: pass
try: result["OutputDirectory"]=model.OutputDirectory
except: pass
try: result["Path_RainScalingFile"]=model.Path_RainScalingFile
except: pass
try: result["Path_SubCatchPolys"]=model.Path_SubCatchPolys
except: pass
try: result["SaveT

In [11]:
now = time.time() # Time after climate collation finished

print("It took: ",(now-then)/60," minutes for climate collation to finish.")

It took:  5.56206404765447  minutes for climate collation to finish.
